In [3]:
# Making a Kuzu graph database of webpages/links that Brad is interested in. 
# Visualized with yFiles

In [4]:
# SECTION 1: DATA CREATION AND LLM CALLS
# --------------------------------------
# This section fetches webpage content, extracts titles, and uses an LLM to categorize and extract keywords,
# producing the final links_with_metadata.csv file for use in the Kùzu database section.


In [5]:
# All imports

import pandas as pd

from pydantic import BaseModel, Field
import json
from ollama import Client

import requests
from bs4 import BeautifulSoup

import kuzu
import os
from collections import Counter

import warnings

import shutil

In [6]:

# Cell 1: Generate links.csv with only URLs
import pandas as pd
links_data = [
    {"url": "https://speakerdeck.com/gaelvaroquaux/open-source-software-how-to-live-long-and-go-far"},
    {"url": "https://medium.com/@seanjtaylor/a-personal-retrospective-on-prophet-f223c2378985"},
    {"url": "https://github.com/jackboyla/GLiREL"},
    {"url": "https://www.latencyconf.io/sessions/pandas-should-go-extinct"},
    {"url": "https://orbae.adastra.eco/"},
    {"url": "https://cambridge-intelligence.com/mapweave/"},
    {"url": "https://github.com/yWorks/yfiles-jupyter-graphs-for-sparql"},
    {"url": "https://holonetgalacticmap-frontend.vercel.app/sparql"},
    {"url": "https://arxiv.org/pdf/2502.13025"},
    {"url": "https://medium.com/eqtventures/knowledge-graph-s-and-llm-based-ontologies-have-a-very-good-shot-at-unlocking-genai-in-production-1b167533ef63"},
    {"url": "https://www.mongodb.com/blog/post/supercharge-ai-data-management-knowledge-graphs"},
    {"url": "https://enterprise-knowledge.com/the-resource-description-framework-rdf/"},
    {"url": "https://medium.com/oracledevs/validating-graph-data-with-shacl-using-oracle-rdf-graph-adapter-for-eclipse-rdf4j-09327042f530"},
    {"url": "https://blog.kuzudb.com/post/kuzu-wasm-rag/"},
    {"url": "https://blog.kuzudb.com/post/unstructured-data-to-graph-baml-kuzu/"},
    {"url": "https://medium.com/@mcgeehan/building-a-hybrid-vector-search-database-with-arrow-and-duckdb-07ebc049bc1f"},
    {"url": "https://github.com/Manirevuri/arangodb-hackathon"},
    {"url": "https://medium.com/@tomzeppenfeldt/querying-an-erp-using-an-ai-configuration-in-graphileon-c667f21f5b51"},
    {"url": "https://2024.connected-data.london/speakers/urbashi-mitra/"},
    {"url": "https://darrendevitt.com/all-fhir-concepts-can-be-explained-simply/"},
    {"url": "https://medium.com/@samschifman/rag-on-fhir-with-knowledge-graphs-04d8e13ee96e"},
    {"url": "https://www.sciencedirect.com/science/article/pii/S1532046422002064"},
    {"url": "https://medium.com/enterprise-rag/why-lawyers-are-uniquely-suited-to-work-with-llms-bcc66d3dce98"},
    {"url": "https://link.springer.com/journal/10506"},
    {"url": "https://www.openownership.org/en/blog/lessons-from-building-a-prototype-single-search-tool-for-beneficial-ownership-registers/"},
    {"url": "https://www.occrp.org/en/project/cyprus-confidential/billionaire-roman-abramovichs-company-set-up-fake-superyacht-chartering-scheme-in-apparent-attempt-to-evade-millions-in-taxes"},
    {"url": "https://www.occrp.org/en/project/the-azerbaijani-laundromat/the-contract-factory-inside-danske-bank-estonias-money-laundering-machine"},
    {"url": "https://www.occrp.org/en/project/the-azerbaijani-laundromat/the-raw-data"},
    {"url": "https://discuss.opensanctions.org/"},
    {"url": "https://blog.opencorporates.com/2025/02/13/getting-started-with-the-opencorporates-api/"},
    {"url": "https://www.buzzsprout.com/242645/episodes/16799543"},
    {"url": "https://www.unodc.org/unodc/en/data-and-analysis/tip-studies.html"},
    {"url": "https://www.which.co.uk/news/article/scam-empire-inside-the-275m-fraud-call-centre-operations-aP3Kc4c9HWd7"},
    {"url": "https://www.occrp.org/en/project/scam-empire/scam-operations-relied-on-third-party-marketing-companies-for-steady-stream-of-potential-victims"},
    {"url": "https://github.com/DAD-CDM/dad-cdm-tsc/blob/main/DAD-CDM-Key-Findings-202502.md"},
    {"url": "https://bods-data.openownership.org/source/gleif_version_0_4/"},
    {"url": "https://eiti.org/using-eiti-data"},
    {"url": "https://mweiti.gov.mw/index.php/reports/details/761"},
    {"url": "https://globalenergymonitor.org/projects/global-energy-ownership-tracker/"},
    {"url": "https://www.moodys.com/web/en/us/kyc/resources/insights/how-ai-is-enhancing-ubo-discovery-and-support-reporting-requirements-globally.html"},
    {"url": "https://oecdstatistics.blog/2019/07/04/the-adima-database-on-multinational-enterprises/"},
    {"url": "https://oecdstatistics.blog/2025/02/21/monitoring-multinational-enterprises-how-the-oecd-and-unsd-are-harnessing-open-data/"},
    {"url": "https://www.taxobservatory.eu/publication/the-end-of-londongrad-ownership-transparency-and-offshore-investment-in-real-estate/"},
    {"url": "https://www.future-fis.com/uploads/3/7/9/4/3794525/ffis_a_new_era_of_private_sector_collaboration_to_detect_economic_crime_-_policy_discussion_paper_-_march_2025_-_final.pdf"},
    {"url": "https://www.kharon.com/brief/outbound-investment-rules-china-hong-kong"},
    {"url": "https://www.gao.gov/products/gao-25-107403"},
    {"url": "https://hoeringsportalen.dk/Hearing/Details/69602"},
    {"url": "https://ubm.se/publikationer/publikationer/2025-03-13-kunskapsrapport---avancerade-angrepp-mot-valfardssystemen"},
    {"url": "https://senzing.com/daniel-silva-prudential-senzing-global-user-conference/"},
    {"url": "https://graphaware.com/resources/streamlining-criminal-assets-confiscation/"},
    {"url": "https://graphaware.com/law-enforcement/"},
    {"url": "https://graphaware.com/blog/aml-investigations-detecting-risk-transactions/"},
    {"url": "https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5120765"},
    {"url": "https://www.powermag.com/software-hardware-innovation-all-needed-to-upgrade-the-power-grid/"},
    {"url": "https://wattclarity.com.au/articles/2025/02/nemde-nightmares-parallel-pathways-and-clashing-constraints/"},
    {"url": "https://mahasldc.in/wp-content/reports/other/Report_on_Optimizing_Power_Despatch.pdf"},
    {"url": "https://github.com/kyribaker/7bus_LMPs"},
    {"url": "https://www.ercot.com/content/cdr/html/real_time_system_conditions.html"},
    {"url": "https://www.abc.net.au/news/2024-10-13/australian-coal-plant-in-extraordinary-survival-experiment/104461504"},
    {"url": "https://www.nrdc.org/resources/uneconomic-coal-costs-miso-ratepayers-1-billion-and-curtails-400-mw-wind"},
    {"url": "https://blog.gridstatus.io/curtailment/"},
    {"url": "https://blog.gridstatus.io/spp-expansion-west/"},
    {"url": "https://www.eia.gov/outlooks/steo/"}
]

df = pd.DataFrame(links_data)
csv_path = "links.csv"
df.to_csv(csv_path, index=False)
print(f"CSV saved to {csv_path}")
print(f"Total links: {len(df)}")


CSV saved to links.csv
Total links: 63


In [7]:
# Cell 2.1: Fetch webpage content and titles
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings("ignore")

def fetch_webpage_content(url, retries=2):
    for attempt in range(retries):
        try:
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            text_elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
            text = ' '.join(element.get_text(strip=True) for element in text_elements)
            if len(text) < 100:
                print(f"Insufficient content for {url}: {len(text)} characters")
                return "", ""
            title = soup.find('title').text if soup.find('title') else url
            return text[:5000], title[:255]
        except Exception as e:
            print(f"Attempt {attempt+1} failed for {url}: {e}, content length: {len(response.text) if 'response' in locals() else 0}")
            if attempt == retries - 1:
                return "", url
    return "", url

csv_path = "links.csv"
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"{csv_path} not found")
links_df = pd.read_csv(csv_path)

links_df["title"] = ""
links_df["content"] = ""
for index, row in links_df.iterrows():
    url = row['url']
    print(f"Fetching content and title for {url}...")
    content, title = fetch_webpage_content(url)
    links_df.at[index, 'content'] = content
    links_df.at[index, 'title'] = title

links_df.to_csv("links_with_raw_content.csv", index=False)
print(f"Raw content and titles saved to links_with_raw_content.csv")
print(f"Processed {len(links_df)} URLs")

Fetching content and title for https://speakerdeck.com/gaelvaroquaux/open-source-software-how-to-live-long-and-go-far...
Fetching content and title for https://medium.com/@seanjtaylor/a-personal-retrospective-on-prophet-f223c2378985...
Fetching content and title for https://github.com/jackboyla/GLiREL...
Fetching content and title for https://www.latencyconf.io/sessions/pandas-should-go-extinct...
Fetching content and title for https://orbae.adastra.eco/...
Fetching content and title for https://cambridge-intelligence.com/mapweave/...
Fetching content and title for https://github.com/yWorks/yfiles-jupyter-graphs-for-sparql...
Fetching content and title for https://holonetgalacticmap-frontend.vercel.app/sparql...
Insufficient content for https://holonetgalacticmap-frontend.vercel.app/sparql: 0 characters
Fetching content and title for https://arxiv.org/pdf/2502.13025...
Insufficient content for https://arxiv.org/pdf/2502.13025: 0 characters
Fetching content and title for https://medium.

In [8]:
# Cell 2.2: Clean fetched content with Ollama (Mistral 7B) using Pydantic
import pandas as pd
import requests
import json
from pydantic import BaseModel, Field

class ContentClassification(BaseModel):
    garbage_text: str = Field(..., description="Irrelevant content (e.g., ads, navigation, boilerplate)", min_length=0)
    cleaned_content: str = Field(..., description="Main meaningful content, cleaned and formatted", min_length=0)
    unsure_content: str = Field(..., description="Content that is ambiguous or unclear", min_length=0)

def clean_content_with_llm(raw_content: str, model: str = "mistral:7b-instruct-v0.3-q4_0", 
                          endpoint: str = "http://localhost:11434/api/chat") -> tuple[str, str, str]:
    if not isinstance(raw_content, str) or not raw_content or len(raw_content.strip()) < 100:
        return "", "", ""
    prompt = (
        "You are an expert at cleaning noisy text data. Analyze the following text and categorize it into three parts: "
        "1. 'garbage_text': Irrelevant content (e.g., advertisements, navigation menus, boilerplate, footers). "
        "2. 'cleaned_content': The main meaningful content, reformatted clearly (e.g., paragraphs or bullet points). "
        "3. 'unsure_content': Text that is ambiguous or unclear if it belongs to the main content or garbage. "
        "Return a JSON object with these three fields, wrapped in triple backticks (```json\n{}\n```). "
        "Do not add or invent information; only categorize and reformat the input. "
        "Example:\n"
        "```json\n"
        "{\n"
        "  \"garbage_text\": \"Navbar: Home | About | Contact\",\n"
        "  \"cleaned_content\": \"This is the main article content.\",\n"
        "  \"unsure_content\": \"Posted by Admin\"\n"
        "}\n"
        "```\n"
        "Raw text:\n"
        f"{raw_content[:5000]}\n"
    )
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 2048,
        "temperature": 0.3
    }
    try:
        response = requests.post(endpoint, json=payload, timeout=30)
        response.raise_for_status()
        raw_response = response.json()["message"]["content"].strip()
        if raw_response.startswith('```json'):
            raw_response = raw_response[7:].rsplit('```', 1)[0].strip()
        result = json.loads(raw_response)
        classification = ContentClassification.model_validate(result)
        return classification.garbage_text, classification.cleaned_content, classification.unsure_content
    except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError, ValueError) as e:
        print(f"Error processing content: {e}")
        return "", raw_content, ""

links_df = pd.read_csv("links_with_raw_content.csv")
cleaning_log = []
for index, row in links_df.iterrows():
    url = row['url']
    print(f"Cleaning content for {url}...")
    content = row['content']
    garbage, cleaned, unsure = clean_content_with_llm(content)
    links_df.at[index, 'content'] = cleaned
    cleaning_log.append({"url": url, "garbage_text": garbage, "unsure_content": unsure})

links_df.to_csv("links_with_cleaned_content.csv", index=False)
pd.DataFrame(cleaning_log).to_csv("cleaning_log.csv", index=False)
print(f"Cleaned content saved to links_with_cleaned_content.csv")
print(f"Cleaning log saved to cleaning_log.csv")

Cleaning content for https://speakerdeck.com/gaelvaroquaux/open-source-software-how-to-live-long-and-go-far...
Error processing content: Extra data: line 2 column 1 (char 147)
Cleaning content for https://medium.com/@seanjtaylor/a-personal-retrospective-on-prophet-f223c2378985...
Error processing content: Extra data: line 2 column 1 (char 147)
Cleaning content for https://github.com/jackboyla/GLiREL...
Error processing content: Extra data: line 2 column 1 (char 147)
Cleaning content for https://www.latencyconf.io/sessions/pandas-should-go-extinct...
Error processing content: Extra data: line 2 column 1 (char 147)
Cleaning content for https://orbae.adastra.eco/...
Error processing content: Extra data: line 2 column 1 (char 147)
Cleaning content for https://cambridge-intelligence.com/mapweave/...
Error processing content: Extra data: line 2 column 1 (char 147)
Cleaning content for https://github.com/yWorks/yfiles-jupyter-graphs-for-sparql...
Error processing content: Extra data: line 2 c

In [9]:
# Cell 2.3: Filter failed URLs and initialize metadata fields
import pandas as pd

links_df = pd.read_csv("links_with_cleaned_content.csv")
failed_urls = []
for index, row in links_df.iterrows():
    url = row['url']
    content = row['content']
    if not isinstance(content, str) or not content or len(content.strip()) < 100:
        failed_urls.append({"url": url, "reason": "Insufficient content after cleaning"})
    links_df.at[index, 'category'] = ""
    links_df.at[index, 'keyword'] = ""
    links_df.at[index, 'category_explanation'] = ""
    links_df.at[index, 'keyword_explanation'] = ""

failed_df = pd.DataFrame(failed_urls)
if not failed_df.empty:
    failed_df.to_csv("links_please_review.csv", index=False, mode='a', header=not os.path.exists("links_please_review.csv"))
    print(f"Failed URLs appended to links_please_review.csv: {len(failed_df)} URLs")

links_df = links_df[links_df['content'].apply(lambda x: isinstance(x, str) and len(x.strip()) >= 100)]
links_df.to_csv("links_with_content.csv", index=False)
print(f"CSV with titles, cleaned content, and empty metadata fields saved to links_with_content.csv")
print(f"Processed {len(links_df)} valid URLs")

Failed URLs appended to links_please_review.csv: 7 URLs
CSV with titles, cleaned content, and empty metadata fields saved to links_with_content.csv
Processed 56 valid URLs


In [10]:
import pandas as pd
from ollama import Client
from pydantic import BaseModel, Field
import json

ollama_client = Client(host='http://localhost:11434')

# Updated Pydantic model to handle up to three keywords
class ArticleClassification(BaseModel):
    category: str = Field(..., description="The assigned category (2-3 words)", min_length=2, max_length=50)
    keywords: list[str] = Field(..., description="Up to three key terms (1-2 words each)", min_items=1, max_items=3)
    category_explanation: str = Field(..., description="One sentence explaining the category choice", min_length=10, max_length=200)
    keyword_explanations: list[str] = Field(..., description="One sentence per keyword explaining the choice", min_items=1, max_items=3)

suggested_categories = [
    "general tools", "graph technologies", "healthcare data", "ai and legal systems",
    "federated search", "organized crime analysis", "beneficial ownership",
    "financial crime technology", "corporate governance", "power and utilities"
]

def process_with_ollama(content, client, suggested_categories):
    if not content or len(content.strip()) < 100:
        return None
    template = f"""
    You are an expert at categorizing articles and extracting key terms. Analyze the article content and provide a structured JSON output with:
    - "category": A category (2-3 words, from suggested categories: {', '.join(suggested_categories)} or a new one if none fit)
    - "keywords": A list of up to three key terms (1-2 words each, specific and descriptive, e.g., 'knowledge graph', not 'data')
    - "category_explanation": One sentence explaining the category choice
    - "keyword_explanations": A list of one sentence per keyword explaining the choice
    Return ONLY the JSON object, wrapped in triple backticks (```json\n{{}}\n```).
    Example:
    ```json
    {{
        "category": "graph technologies",
        "keywords": ["knowledge graph", "graph database", "semantic search"],
        "category_explanation": "The article discusses graph-based data management.",
        "keyword_explanations": [
            "Knowledge graph is the central concept of the article.",
            "Graph database is discussed as the technology used.",
            "Semantic search is mentioned as an application."
        ]
    }}
    ```
    Content: {content[:2000]}
    """
    try:
        response = client.generate(
            model='mistral:7b-instruct-v0.3-q4_0',
            prompt=template,
            options={"temperature": 0.4}
        )
        raw_response = response['response'].strip()
        if raw_response.startswith('```json'):
            raw_response = raw_response[7:].rsplit('```', 1)[0].strip()
        elif not raw_response.startswith('{'):
            raise ValueError("Unexpected response format from Ollama")
        print(f"Raw LLM response for content: {raw_response[:200]}...")
        result = json.loads(raw_response)
        classification = ArticleClassification.model_validate(result)
        return classification
    except Exception as e:
        print(f"Error processing content: {str(e)}")
        return None

links_df = pd.read_csv("links_with_content.csv")
failed_processing = []
for index, row in links_df.iterrows():
    url = row['url']
    print(f"Processing {url} for categorization and keywords...")
    content = row['content']
    result = process_with_ollama(content, ollama_client, suggested_categories)
    if result:
        links_df.at[index, 'category'] = result.category
        # Store keywords as a comma-separated string
        links_df.at[index, 'keyword'] = ", ".join(result.keywords)
        links_df.at[index, 'category_explanation'] = result.category_explanation
        # Store keyword explanations as a semicolon-separated string
        links_df.at[index, 'keyword_explanation'] = "; ".join(result.keyword_explanations)
    else:
        links_df.at[index, 'category'] = "uncategorized"
        links_df.at[index, 'keyword'] = "none"
        links_df.at[index, 'category_explanation'] = "Failed to process content."
        links_df.at[index, 'keyword_explanation'] = "Failed to extract keywords."
        failed_processing.append({"url": url, "reason": "Failed to generate meaningful category or keywords"})

failed_processing_df = pd.DataFrame(failed_processing)
if not failed_processing_df.empty:
    failed_processing_df.to_csv("failed_processing.csv", index=False)
    print(f"Failed processing saved to failed_processing.csv: {len(failed_processing_df)} URLs")

links_df.to_csv("links_with_metadata.csv", index=False)
print("Updated CSV with titles, content, categories, keywords, and explanations saved to links_with_metadata.csv")
unique_categories = links_df['category'].unique().tolist()
# Calculate keyword distribution
keyword_list = []
for keywords in links_df['keyword']:
    if keywords and keywords != "none":
        keyword_list.extend([k.strip() for k in keywords.split(",")])
keyword_distribution = pd.Series(keyword_list).value_counts()
print(f"Unique categories: {unique_categories}")
print(f"Keyword distribution:\n{keyword_distribution}")
links_df.sample(5)

Processing https://speakerdeck.com/gaelvaroquaux/open-source-software-how-to-live-long-and-go-far for categorization and keywords...
Raw LLM response for content: {
        "category": "general tools",
        "keywords": ["open-source software", "Python", "community"],
        "category_explanation": "The article provides a guide to building open-source softwa...
Processing https://medium.com/@seanjtaylor/a-personal-retrospective-on-prophet-f223c2378985 for categorization and keywords...
Raw LLM response for content: {
        "category": "ai and legal systems",
        "keywords": ["forecasting library", "Prophet", "time-series"],
        "category_explanation": "The article discusses an AI tool (Prophet) used fo...
Processing https://github.com/jackboyla/GLiREL for categorization and keywords...
Raw LLM response for content: {
        "category": "ai and legal systems",
        "keywords": ["relation extraction", "named entity recognition", "spacy"],
        "category_explanation": 

,url,title,content,category,keyword,category_explanation,keyword_explanation
43,https://ubm.se/publikationer/publikationer/202...,Kunskaps­rapport – Avancerade angrepp mot välf...,Till innehåll Dölj navigering Utbetalningsmynd...,organized crime analysis,"fraudulent companies, identity abuse, social s...",The article discusses advanced attacks on welf...,Fraudulent companies is the main actor in the ...
46,https://graphaware.com/law-enforcement/,Connected Data for Criminal Intelligence Analysis,Connected data analytics platform. Explore how...,organized crime analysis,"connected data analytics, graph technology",The article discusses the use of connected dat...,Connected data analytics is the central concep...
37,https://oecdstatistics.blog/2019/07/04/the-adi...,The ADIMA database on Multinational Enterprises,Recent Posts Most Used Categories The ADIMA d...,financial crime technology,"MNEs, taxes, Effective Tax Rate",The article discusses a database (ADIMA) that ...,MNEs are the main subject of the article.; Tax...
41,https://www.gao.gov/products/gao-25-107403,Illicit Finance: Treasury's Initial Safeguards...,U.S. Government Accountability Office Breadcru...,financial crime technology,"beneficial ownership, Financial Crimes Enforce...",The article discusses the U.S. government's ef...,Beneficial owners are a central concept in the...
49,https://www.powermag.com/software-hardware-inn...,"Software, Hardware, Innovation All Needed to U...",POWERis at the forefront of the global power m...,power and utilities,"power grid, enhancements, reliability",The article discusses the power grid and its i...,Power grid is the main subject of the article....


In [11]:

# SECTION 2: KÙZU DATABASE AND VISUALIZATION
# ----------------------------------------
# This section initializes the Kùzu database, populates it with data from links_with_metadata.csv,
# queries for interconnections, and visualizes the graph with yFiles.


In [14]:
# Cell 4: Initialize Kùzu database
import kuzu
import os
import shutil
import pandas as pd

# Database path
db_path = os.path.join("..", "db", "graph_db")

# Delete existing database only if a fresh start is needed
if os.path.exists(db_path):
    shutil.rmtree(db_path)
    print(f"Deleted existing database at {db_path}")

# Create database directory
os.makedirs(db_path, exist_ok=True)
print(f"Ensured database directory at {db_path}")

# Initialize database and connection (persist across cells)
try:
    db = kuzu.Database(db_path)
    conn = kuzu.Connection(db)
except Exception as e:
    print(f"Error initializing Kùzu database at {db_path}: {str(e)}")
    raise

# Create schema if it doesn't exist
try:
    # Check if tables already exist
    result = conn.execute("CALL show_tables() RETURN name")
    existing_tables = []
    while result.has_next():
        row = result.get_next()
        existing_tables.append(row[0])
    print(f"Existing tables: {existing_tables}")

    if "Link" not in existing_tables:
        conn.execute("CREATE NODE TABLE Link(url STRING, category STRING, title STRING, keyword STRING, category_explanation STRING, keyword_explanation STRING, PRIMARY KEY(url))")
        conn.execute("CREATE NODE TABLE Category(name STRING, PRIMARY KEY(name))")
        conn.execute("CREATE NODE TABLE Keyword(name STRING, PRIMARY KEY(name))")
        conn.execute("CREATE REL TABLE BELONGS_TO(FROM Link TO Category)")
        conn.execute("CREATE REL TABLE HAS_KEYWORD(FROM Link TO Keyword)")
        print("Database schema created successfully.")
    else:
        print("Schema already exists, skipping creation.")

    # Verify schema
    result = conn.execute("CALL show_tables() RETURN name")
    tables = []
    while result.has_next():
        row = result.get_next()
        tables.append(row[0])
    if "Link" not in tables:
        raise ValueError("Link table not found in schema.")
    print("Schema verified: Link table exists.")

except Exception as e:
    print(f"Error initializing schema: {str(e)}")
    raise

Deleted existing database at ..\db\graph_db
Ensured database directory at ..\db\graph_db
Existing tables: []
Database schema created successfully.
Schema verified: Link table exists.


In [16]:
# Cell 5: Populate Kùzu database with links, categories, and keywords
import kuzu
import pandas as pd
import os

db_path = os.path.join("..", "db", "graph_db")
csv_path = "links_with_metadata.csv"

try:
    # Validate database and connection
    if not ('db' in globals() and 'conn' in globals()):
        raise ValueError("Database and connection not initialized. Run Cell 4 first.")
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database path {db_path} not found. Run Cell 4 to initialize.")

    # Validate CSV
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"{csv_path} not found. Ensure Section 1 has generated links_with_metadata.csv.")
    links_df = pd.read_csv(csv_path)
    required_columns = ['url', 'category', 'title', 'keyword', 'category_explanation', 'keyword_explanation', 'content']
    missing_columns = [col for col in required_columns if col not in links_df.columns]
    if missing_columns:
        raise ValueError(f"CSV missing required columns: {missing_columns}")

    # Filter out failed links
    links_df = links_df[
        (links_df['content'].apply(lambda x: isinstance(x, str) and len(x.strip()) >= 100)) &
        (links_df['category'] != "uncategorized") &
        (links_df['keyword'] != "none")
    ]
    print(f"Filtered to {len(links_df)} valid links after removing failed entries.")
    if links_df.empty:
        raise ValueError("No valid links remain after filtering. Check Section 1 output.")

    # Verify schema
    result = conn.execute("CALL show_tables() RETURN name")
    tables = []
    while result.has_next():
        row = result.get_next()
        tables.append(row[0])
    if not all(table in tables for table in ["Link", "Category", "Keyword", "BELONGS_TO", "HAS_KEYWORD"]):
        raise ValueError("Required tables or relationships missing. Run Cell 4 to recreate the schema.")

    # Batch insert categories
    for category in links_df['category'].unique():
        if pd.notna(category) and category != "uncategorized":
            category = category.replace("'", "\\'")
            conn.execute(f"MERGE (c:Category {{name: '{category}'}})")

    # Batch insert keywords
    all_keywords = set()
    for keywords in links_df['keyword']:
        if pd.notna(keywords) and keywords != "none":
            all_keywords.update(k.strip() for k in keywords.split(",") if k.strip())
    for keyword in all_keywords:
        if pd.notna(keyword):
            keyword = keyword.replace("'", "\\'")
            conn.execute(f"MERGE (k:Keyword {{name: '{keyword}'}})")

    # Insert links and relationships
    for _, row in links_df.iterrows():
        url = row['url'].replace("'", "\\'")
        category = row['category'].replace("'", "\\'") if pd.notna(row['category']) else "uncategorized"
        title = row['title'].replace("'", "\\'") if pd.notna(row['title']) else ""
        keyword = row['keyword'].replace("'", "\\'") if pd.notna(row['keyword']) else "none"
        category_explanation = row['category_explanation'].replace("'", "\\'") if pd.notna(row['category_explanation']) else ""
        keyword_explanation = row['keyword_explanation'].replace("'", "\\'") if pd.notna(row['keyword_explanation']) else ""
        
        conn.execute(f"""
            MERGE (l:Link {{url: '{url}'}})
            SET l.category = '{category}',
                l.title = '{title}',
                l.keyword = '{keyword}',
                l.category_explanation = '{category_explanation}',
                l.keyword_explanation = '{keyword_explanation}'
        """)
        
        if pd.notna(row['category']) and row['category'] != "uncategorized":
            conn.execute(f"""
                MATCH (l:Link {{url: '{url}'}}), (c:Category {{name: '{category}'}})
                MERGE (l)-[:BELONGS_TO]->(c)
            """)

        if pd.notna(row['keyword']) and row['keyword'] != "none":
            for keyword in row['keyword'].split(","):
                keyword = keyword.strip().replace("'", "\\'")
                if keyword:
                    conn.execute(f"""
                        MATCH (l:Link {{url: '{url}'}}), (k:Keyword {{name: '{keyword}'}})
                        MERGE (l)-[:HAS_KEYWORD]->(k)
                    """)

    print("Kùzu database populated successfully.")
    result = conn.execute("MATCH (l:Link) WHERE l.category = 'uncategorized' RETURN COUNT(l)")
    uncategorized_count = result.get_next()[0]
    print(f"Uncategorized nodes: {uncategorized_count}")  # Should be 0 due to filtering
    if uncategorized_count > 0:
        print("Warning: Uncategorized nodes found despite filtering. Check data in links_with_metadata.csv.")

except Exception as e:
    print(f"Error populating database: {str(e)}")
    raise

Filtered to 55 valid links after removing failed entries.
Kùzu database populated successfully.
Uncategorized nodes: 0


In [17]:
# Cell 6: Query Kùzu for interconnections between categories
import kuzu
import pandas as pd
import os

db_path = os.path.join("..", "db", "graph_db")

try:
    # Validate database and connection
    if not ('db' in globals() and 'conn' in globals()):
        raise ValueError("Database and connection not initialized. Run Cell 4 first.")
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database path {db_path} not found. Run Cell 4 to initialize.")

    # Diagnostic queries
    result = conn.execute("MATCH (l:Link) RETURN COUNT(l)")
    link_count = result.get_next()[0]
    print(f"Total Link nodes: {link_count}")
    if link_count == 0:
        raise ValueError("No Link nodes found. Run Cell 5 to populate the database.")

    result = conn.execute("MATCH (l:Link)-[:HAS_KEYWORD]->(k:Keyword) RETURN COUNT(l)")
    print(f"Links with keywords: {result.get_next()[0]}")
    result = conn.execute("MATCH (l:Link)-[:BELONGS_TO]->(c:Category) RETURN COUNT(l)")
    print(f"Links with categories: {result.get_next()[0]}")

    # Query interconnections
    results = conn.execute("""
        MATCH (l1:Link)-[:HAS_KEYWORD]->(k:Keyword)<-[:HAS_KEYWORD]-(l2:Link),
              (l1)-[:BELONGS_TO]->(c1:Category),
              (l2)-[:BELONGS_TO]->(c2:Category)
        WHERE c1.name <> c2.name AND l1.url < l2.url
        RETURN l1.url, l1.category, l2.url, l2.category, k.name AS shared_keyword
        ORDER BY k.name, l1.category, l2.category
        LIMIT 100
    """)

    interconnections = []
    while results.has_next():
        row = results.get_next()
        interconnections.append({
            "Link1_URL": row[0],
            "Link1_Category": row[1],
            "Link2_URL": row[2],
            "Link2_Category": row[3],
            "Shared_Keyword": row[4]
        })

    interconnections_df = pd.DataFrame(interconnections)
    if not interconnections_df.empty:
        print("Interconnections between categories:")
        print(interconnections_df.to_string(index=False))
    else:
        print("No interconnections found. Verify keywords and categories in Cell 5 output.")

except Exception as e:
    print(f"Error querying database: {str(e)}")
    raise

Total Link nodes: 55
Links with keywords: 156
Links with categories: 55
Interconnections between categories:
                                                                                                                                    Link1_URL             Link1_Category                                                                                                                                     Link2_URL             Link2_Category                       Shared_Keyword
                                                               https://medium.com/@samschifman/rag-on-fhir-with-knowledge-graphs-04d8e13ee96e            healthcare data                                              https://medium.com/enterprise-rag/why-lawyers-are-uniquely-suited-to-work-with-llms-bcc66d3dce98       ai and legal systems Retrieval Augmented Generation (RAG)
                                                                                          https://mweiti.gov.mw/index.php/reports/details/761

In [18]:
# Cell 7: Clean Kùzu graph and visualize with yFiles
import kuzu
import os
import pandas as pd
try:
    from yfiles_jupyter_graphs import GraphWidget
except ImportError:
    raise ImportError("Install yfiles-jupyter-graphs: pip install yfiles-jupyter-graphs")

db_path = os.path.join("..", "db", "graph_db")

def clean_database():
    try:
        # Validate database and connection
        if not ('db' in globals() and 'conn' in globals()):
            raise ValueError("Database and connection not initialized. Run Cell 4 first.")
        if not os.path.exists(db_path):
            raise FileNotFoundError(f"Database path {db_path} not found.")

        # Verify no uncategorized links exist (due to filtering in Cell 5)
        result = conn.execute("MATCH (l:Link) WHERE l.category = 'uncategorized' RETURN l.url, l.title")
        uncategorized = []
        while result.has_next():
            row = result.get_next()
            uncategorized.append({"url": row[0], "title": row[1] or "No Title"})
        if uncategorized:
            print(f"Warning: Found {len(uncategorized)} uncategorized links despite filtering.")
            pd.DataFrame(uncategorized).to_csv("uncategorized_links.csv", index=False)

        # Clean orphaned nodes
        conn.execute("MATCH (c:Category) WHERE NOT (c)<-[:BELONGS_TO]-() DELETE c")
        conn.execute("MATCH (k:Keyword) WHERE NOT (k)<-[:HAS_KEYWORD]-() DELETE k")
        print("Orphaned nodes cleaned.")

    except Exception as e:
        print(f"Error cleaning database: {str(e)}")
        raise

try:
    clean_database()

    # Validate data
    result = conn.execute("MATCH (l:Link) RETURN COUNT(l)")
    link_count = result.get_next()[0]
    print(f"Total Link nodes: {link_count}")
    if link_count == 0:
        raise ValueError("No Link nodes found. Run Cell 5 to populate the database.")

    nodes = []
    edges = []

    # Add Link nodes
    result = conn.execute("MATCH (l:Link) RETURN l.url, l.category, l.title, l.keyword LIMIT 50")
    while result.has_next():
        row = result.get_next()
        nodes.append({
            "id": row[0],
            "properties": {
                "type": "Link",
                "label": row[2] or row[0],
                "category": row[1],
                "keywords": row[3]  # Include keywords for display
            }
        })

    # Add Category nodes
    result = conn.execute("MATCH (c:Category) RETURN c.name")
    while result.has_next():
        row = result.get_next()
        nodes.append({"id": row[0], "properties": {"type": "Category", "label": row[0]}})

    # Add Keyword nodes
    result = conn.execute("MATCH (k:Keyword) RETURN k.name")
    while result.has_next():
        row = result.get_next()
        nodes.append({"id": row[0], "properties": {"type": "Keyword", "label": row[0]}})

    # Add BELONGS_TO edges
    result = conn.execute("MATCH (l:Link)-[:BELONGS_TO]->(c:Category) RETURN l.url, c.name")
    while result.has_next():
        row = result.get_next()
        edges.append({"start": row[0], "end": row[1], "properties": {"type": "BELONGS_TO"}})

    # Add HAS_KEYWORD edges
    result = conn.execute("MATCH (l:Link)-[:HAS_KEYWORD]->(k:Keyword) RETURN l.url, k.name")
    while result.has_next():
        row = result.get_next()
        edges.append({"start": row[0], "end": row[1], "properties": {"type": "HAS_KEYWORD"}})

    # Visualize
    w = GraphWidget()
    w.nodes = nodes
    w.edges = edges

    def node_mappings(node):
        node_type = node['properties']['type']
        if node_type == "Link":
            return {"color": "#1E90FF", "shape": "rectangle"}
        elif node_type == "Category":
            return {"color": "#32CD32", "shape": "ellipse"}
        elif node_type == "Keyword":
            return {"color": "#FF4500", "shape": "triangle"}
        return {"color": "#808080", "shape": "circle"}

    w.node_mappings = node_mappings

    def edge_mappings(edge):
        edge_type = edge['properties']['type']
        if edge_type == "BELONGS_TO":
            return {"color": "#FFD700"}
        elif edge_type == "HAS_KEYWORD":
            return {"color": "#9932CC"}
        return {"color": "#000000"}

    w.node_label_mapping = lambda node: node['properties']['label']
    w.set_graph_layout("organic")
    w.show()
    print("Graph visualization completed.")

except Exception as e:
    print(f"Error visualizing graph: {str(e)}")
    raise

Orphaned nodes cleaned.
Total Link nodes: 55


GraphWidget(layout=Layout(height='800px', width='100%'))

Graph visualization completed.
